In [4]:
import glob
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
current_dir = os.getcwd()
root_dir = os.path.dirname(current_dir)
stimulated_data_file_path = os.path.join(root_dir,"data", "stimulated_data.csv")
src_code = os.path.join(root_dir, "oil-wells-data-scraper", "src")
# os.chdir(src_code)
sys.path.append(src_code)

In [14]:
df = pd.read_csv(stimulated_data_file_path)
df["date_stimulated"] = pd.to_datetime(df["date_stimulated"], errors="coerce")
df.replace({pd.NA: None, np.nan: None}, inplace=True)
# df.fillna(value=None, inplace=True)
df

C:\Users\shahk\AppData\Local\Temp\ipykernel_5752\4021639085.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date_stimulated"] = pd.to_datetime(df["date_stimulated"], errors="coerce")


,file_id,api_no,date_stimulated,stimulated_formation,top_ft,bottom_ft,stimulation_stages,volume,volume_units,type_treatment,lbs_proppant,maximum_treatment_pressure_psi,maximum_treatment_rate_bbls_per_min
0,W22099,33-053-03911,2013-08-31 00:00:00,Bakken,11126.0,21220.0,36.0,100130.0,Barrels,Sand Frac,3649776.0,9294.0,39.0
1,W22221,33-053-03937,2013-09-07 00:00:00,Bakken,11117.0,21122.0,36.0,90354.0,Barrels,Sand Frac,3765029.0,8782.0,37.5
2,W22731,33-053-04069,2021-07-07 00:00:00,Middle Bakken,11126.0,21145.0,35.0,40332.0,Barrels,Sand Frac,3630945.0,7548.0,34.0
3,W22740,33-053-04071,2012-09-08 00:00:00,Bakken,11080.0,21140.0,36.0,80480.0,Barrels,Sand Frac,3600530.0,9705.0,39.5
4,W23230,33-053-04211,2012-10-26 00:00:00,ThreeForks,11101.0,20482.0,36.0,92216.0,Barrels,Sand Frac,3700348.0,9214.0,42.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,W23365,33-105-02725,2014-12-14 00:00:00,ThreeForks,10865.0,22585.0,38.0,72511.0,Barrels,Sand Frac,3764730.0,8557.0,23.0
72,W20863,33-053-03608,2011-06-11 00:00:00,Bakken,10162.0,20440.0,36.0,74475.0,Barrels,Sand Frac,4519292.0,8877.0,43.6
73,W90329,33-053-90329,None,None,None,None,None,None,Barrels,Sand Frac,0.0,0.0,0.0
74,W21266,33-053-03703,2012-11-01 00:00:00,Bakken,10952.0,20850.0,13.0,1822380.0,Gallons,Sand Frac,1450600.0,8842.0,45.5


In [15]:
# Filter duplicate rows by api_no
df_filtered = df[df.duplicated(subset=['api_no'], keep=False)]

# Print the filtered DataFrame
df_filtered

,file_id,api_no,date_stimulated,stimulated_formation,top_ft,bottom_ft,stimulation_stages,volume,volume_units,type_treatment,lbs_proppant,maximum_treatment_pressure_psi,maximum_treatment_rate_bbls_per_min


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   file_id                              76 non-null     object
 1   api_no                               76 non-null     object
 2   date_stimulated                      70 non-null     object
 3   stimulated_formation                 71 non-null     object
 4   top_ft                               70 non-null     object
 5   bottom_ft                            71 non-null     object
 6   stimulation_stages                   70 non-null     object
 7   volume                               71 non-null     object
 8   volume_units                         73 non-null     object
 9   type_treatment                       73 non-null     object
 10  lbs_proppant                         70 non-null     object
 11  maximum_treatment_pressure_psi       70 non-nul

In [22]:
from database import get_db
db=get_db()

In [58]:
from sqlalchemy import text

c_apino = [i[0] for i in db.execute(text("SELECT api_no from clean_well_data")).fetchall()]

# c_apino

In [56]:
unique_ids = [id for id in df.api_no.to_list() if id not in c_apino]
unique_ids

[]

In [20]:
from typing import List
import schema
import model
from sqlalchemy.orm import Session


# def bulk_ingest_stimulated_data(requests: List[schema.WellTreatment], db: Session):
#     new_docs = []

#     for np in requests:
#         new_post = model.WellTreatmentSQLModel(**np.model_dump())
#         new_docs.append(new_post)

#     db.bulk_save_objects(new_docs)
#     db.commit()

#     print(f"Successfully added '{len(new_docs)}' documents to the database")
#     return new_docs

# clean_documents=[]
# for doc in tqdm(df.to_dict(orient="records")):
#     data = schema.WellTreatment(**doc)
#     if data.api_no is not None:
#         clean_documents.append(data)
# bulk_ingest_stimulated_data(clean_documents, db)
# clean_documents

In [60]:
from sqlalchemy import select
from model import CleanWellDataModel, WellTreatmentSQLModel


data = (
    db.query(
        CleanWellDataModel,
        WellTreatmentSQLModel.date_stimulated,
        WellTreatmentSQLModel.stimulated_formation,
        WellTreatmentSQLModel.top_ft,
        WellTreatmentSQLModel.bottom_ft,
        WellTreatmentSQLModel.stimulation_stages,
        WellTreatmentSQLModel.volume,
        WellTreatmentSQLModel.volume_units,
        WellTreatmentSQLModel.type_treatment,
        WellTreatmentSQLModel.lbs_proppant,
        WellTreatmentSQLModel.maximum_treatment_pressure_psi,
        WellTreatmentSQLModel.maximum_treatment_rate_bbls_per_min
    )
    .outerjoin(WellTreatmentSQLModel, CleanWellDataModel.api_no == WellTreatmentSQLModel.api_no)
    .all()
)

result_dicts = []
for row in data:
    clean_well_data_dict = row[0].__dict__
    treatment_data_dict = {
        'date_stimulated': row[1],
        'stimulated_formation': row[2],
        'top_ft': row[3],
        'bottom_ft': row[4],
        'stimulation_stages': row[5],
        'volume': row[6],
        'volume_units': row[7],
        'type_treatment': row[8],
        'lbs_proppant': row[9],
        'maximum_treatment_pressure_psi': row[10],
        'maximum_treatment_rate_bbls_per_min': row[11]
    }
    result_dicts.append({**clean_well_data_dict, **treatment_data_dict})

# Fetch the results
for row in result_dicts:
    print(row)
print(len(result_dicts))

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000023FD6309FC0>, 'latest_barrels_of_oil_produced': None, 'api_no': '33-053-00605', 'latitude': 47.3462, 'longitude': -103.554, 'well_name': 'Wrrmu 4-28', 'well_type': 'Oil & Gas', 'latest_mcf_of_gas_produced': None, 'closest_city': 'Medora', 'county': 'McKenzie County, ND', 'link': 'https://www.drillingedge.com/north-dakota/mckenzie-county/wells/wrrmu-4-28/33-053-00605', 'operator': 'SM Energy Company', 'well_status': 'Plugged and Abandoned', 'date_stimulated': datetime.datetime(2015, 7, 13, 0, 0), 'stimulated_formation': 'Bakken', 'top_ft': 18852.0, 'bottom_ft': 20390.0, 'stimulation_stages': 6.0, 'volume': 19114.0, 'volume_units': 'Barrels', 'type_treatment': 'Sand Frac', 'lbs_proppant': 577940.0, 'maximum_treatment_pressure_psi': 8860.0, 'maximum_treatment_rate_bbls_per_min': 36.7}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000023FD63099C0>, 'latest_barrels_of_oil_produced': None, '